# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

# Задание 1

In [113]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

KEYWORDS = ['парсинг', 'python']

In [114]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

posts = soup.find_all('article', class_='post post_preview')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/Audioman/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/7fd/66e/5a3/7fd66e5a3b645f7ecdd9c99e5893c15d.png" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">Audioman</span>
 </a>
 <span class="post__time">сегодня в 21:20</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/audiomania/blog/540790/">Слушать, чтобы работать — агрегатор лоу-фай стримов, шумовой микшер и плеер для бесплатной музыки</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/audiomania/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Блог к

In [117]:
title_all = []
post_time_all = []
post_link_all = []

for post in posts:
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
        hub_lower = hub.text.lower()
        if any([hub_lower in desired for desired in KEYWORDS]):
            title_element = post.find('a', class_='post__title_link')
            post_time = post.find('span', class_='post__time').text
            post_time_all += [post_time]
            title_all += [title_element.text]
            post_link_all += [title_element.attrs.get('href')]

kw = {'post time': pd.Series(post_time_all), 'title': pd.Series(title_all), 'link' : pd.Series(post_link_all)}
pd.DataFrame(kw)

,post time,title,link
0,сегодня в 15:21,"Pythonista. Привет, Python",https://habr.com/ru/post/543650/


# Задание 2

In [202]:
import json
import time
import re

In [631]:
EMAILS = ['xxx@yandex.ru', 'tasey14@gmail.com']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
Headers = {'Vaar-Version': '0', 
           'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
           'Vaar-Header-App-Product': 'hackcheck-web-avast',
           'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
           'Vaar-Header-App-Build-Version': '1.0.0'
          }

In [632]:
df = pd.DataFrame()

for email in EMAILS:
    params = {'emailAddresses' : [email]}
    req_avast = requests.post(URL, json=params, headers=Headers)
    info_1 = req_avast.text
    site = re.findall(r'site":"([^"]+)', info_1)
    data = re.findall(r'publishDate":"(\d{4}-\d{2}-\d{2})', info_1)
    description = re.findall(r'description":"([^"]+)', info_1)
    req_breashed = {'почта': (email),
            'дата утечки': pd.Series(data),
            'источник утечки' : pd.Series(site),
            'описание утечки' : pd.Series(description)}
    info_2 = pd.DataFrame(req_breashed)
    print(info_2)
    df.append(info_2, ignore_index = True)
    time.sleep(0.3)
df

            почта дата утечки   источник утечки  \
0   xxx@yandex.ru  2019-03-28  verifications.io   
1   xxx@yandex.ru  2020-05-21            vk.com   
2   xxx@yandex.ru  2020-05-21    bitrewards.com   
3   xxx@yandex.ru  2020-12-10          job12.eu   
4   xxx@yandex.ru  2020-12-03    worldofauto.ru   
5   xxx@yandex.ru  2020-04-09         vkmix.com   
6   xxx@yandex.ru  2020-12-17           e-ga.ru   
7   xxx@yandex.ru  2020-12-17        tcl.y17.su   
8   xxx@yandex.ru  2020-11-05      vimeworld.ru   
9   xxx@yandex.ru  2020-12-10        shedevr.ru   
10  xxx@yandex.ru  2019-05-23   livejournal.com   

                                      описание утечки  
0   Big data e-mail verification platform verifica...  
1   At some time in 2020, the Russian social netwo...  
2   In April 2020, the e-commerce platform BitRewa...  
3   In November 2020, a collection of over 23,000 ...  
4   In November 2020, a collection of over 23,000 ...  
5   At an unconfirmed date, the Russian social med.

""
